In [0]:
# how to use autoloader
# autoloader is built on top of spark streaming
# bookings table - or file
# autoloader incrementally reads data
# and stores schema in the checkpoint.
# if the data schema changes, autoloader will update the cache.
# cache is not refreshed at every read, only when data is written.





### Incremental Data Ingestion

<!-- either create dedicated volume or dedicated schema for different layers. -->


In [0]:
# %sql
# CREATE VOLUME workspace.raw.bronze

or

In [0]:
# %sql
# CREATE SCHEMA workspace.bronze

In [0]:
# %sql
# CREATE SCHEMA workspace.silver

In [0]:
# %sql
# CREATE SCHEMA workspace.gold

In [0]:
# %sql
# CREATE VOLUME workspace.bronze.bronze_volume

In [0]:
# %sql

# CREATE VOLUME workspace.silver.silver_volume

In [0]:
# %sql

# CREATE VOLUME workspace.gold.gold_volume

In [0]:
dbutils.widgets.text('src', '')

In [0]:
src_value = dbutils.widgets.get('src')
assert src_value, "Missing job parameter: src"
print("SRC =", src_value)

In [0]:
# dbutils.fs.rm(f"/Volumes/workspace/bronze/bronze_volume/{src_value}/schema", True)

In [0]:
# we need checkpoint location in spark streaming (not batch processing) to store schema because it is needed to infer schema
# because it's easier to maintain and easy to manage in production scenarios.


df = spark.readStream.format('cloudFiles')\
    .option('cloudFiles.format', 'csv')\
    .option('cloudFiles.schemaLocation', f'/Volumes/workspace/bronze/bronze_volume/{src_value}/schema')\
    .option('cloudFiles.schemaEvolutionMode', 'rescue')\
    .load(f'/Volumes/workspace/raw/rawvolume/rawdata/{src_value}/')

    # analyze what's resume schema evolution mode
    # 

In [0]:
df.writeStream.format('delta')\
    .outputMode('append')\
    .trigger(once=True)\
    .option('checkpointLocation', 
            f'/Volumes/workspace/bronze/bronze_volume/{src_value}/checkpoint')\
    .option('path', f'/Volumes/workspace/bronze/bronze_volume/{src_value}/data')\
    .start()
    

In [0]:
%sql
SELECT * FROM delta.`/Volumes/workspace/bronze/bronze_volume/customers/data`